In [ ]:
pip install -U sentence-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 KB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 38.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 31.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 18.5 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=3360ada168910f8ac8ac816bd0a8fb083032249f4f7c81fcdbbac9ab67c0735e
  Stored in directory: /root/.cache/pip/wheels/5e/6f/8c/d88aec621f3f542d26fac0342bef5e693335d125f4e54aeffe
Successfully built sentence-transformers


In [ ]:
import nltk
import pandas as pd
from sentence_transformers import SentenceTransformer
from nltk.cluster import KMeansClusterer
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
with open("/content/gdrive/MyDrive/project/story.txt") as f:
  body = f.read()

In [ ]:
body

'\nIN WHICH PHILEAS FOGG AND PASSEPARTOUT ACCEPT EACH OTHER,\nTHE ONE AS MASTER, THE OTHER AS MAN\n\n\nMr. Phileas Fogg lived, in 1872, at No. 7, Saville Row, Burlington\nGardens, the house in which Sheridan died in 1814.  He was one of the\nmost noticeable members of the Reform Club, though he seemed always to\navoid attracting attention; an enigmatical personage, about whom little\nwas known, except that he was a polished man of the world.  People said\nthat he resembled Byron--at least that his head was Byronic; but he was\na bearded, tranquil Byron, who might live on a thousand years without\ngrowing old.\n\nCertainly an Englishman, it was more doubtful whether Phileas Fogg was\na Londoner.  He was never seen on \'Change, nor at the Bank, nor in the\ncounting-rooms of the "City"; no ships ever came into London docks of\nwhich he was the owner; he had no public employment; he had never been\nentered at any of the Inns of Court, either at the Temple, or Lincoln\'s\nInn, or Gray\'s In

In [ ]:
from sentence_transformers import SentenceTransformer

In [ ]:
	model = SentenceTransformer('stsb-roberta-base')

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
sentences=nltk.sent_tokenize(body)
 
# strip leading and trailing spaces
sentences = [sentence.strip() for sentence in sentences]

In [ ]:
data = pd.DataFrame(sentences)
 
data.columns=['sentence']

In [ ]:
def get_sentence_embeddings(sentence):
    embedding = model.encode([sentence])
    return embedding[0]

In [ ]:
data['embeddings']=data['sentence'].apply(get_sentence_embeddings)

In [ ]:
NUM_CLUSTERS=10
 
iterations=25
 
X = np.array(data['embeddings'].tolist())
 
kclusterer = KMeansClusterer(
        NUM_CLUSTERS, distance=nltk.cluster.util.cosine_distance,
        repeats=iterations,avoid_empty_clusters=True)
 
assigned_clusters = kclusterer.cluster(X, assign_clusters=True)

In [ ]:
data['cluster']=pd.Series(assigned_clusters, index=data.index)
data['centroid']=data['cluster'].apply(lambda x: kclusterer.means()[x])

In [ ]:
from scipy.spatial import distance_matrix
def distance_from_centroid(row):
    #type of emb and centroid is different, hence using tolist below
    return distance_matrix([row['embeddings']], [row['centroid'].tolist()])[0][0]
 
data['distance_from_centroid'] = data.apply(distance_from_centroid, axis=1)

In [ ]:
	
summary=' '.join(data.sort_values('distance_from_centroid',ascending = True).groupby('cluster').head(1).sort_index()['sentence'].tolist())

In [ ]:
print(summary)

Phileas Fogg was a member of the Reform, and that was all. He was recommended by the Barings, with whom he had an open credit. He was, in short, the least
communicative of men. His daily habits were quite open
to observation; but whatever he did was so exactly the same thing that
he had always done before, that the wits of the curious were fairly
puzzled. He often won at this game, which, as a
silent one, harmonised with his nature; but his winnings never went
into his purse, being reserved as a fund for his charities. He lived alone in his house in
Saville Row, whither none penetrated. When he breakfasted or dined all the resources of the club--its
kitchens and pantries, its buttery and dairy--aided to crowd his table
with their most succulent stores; he was served by the gravest waiters,
in dress coats, and shoes with swan-skin soles, who proffered the
viands in special porcelain, and on the finest linen; club decanters,
of a lost mould, contained his sherry, his port, and his
cinnam

In [ ]:
import spacy
nlp = spacy.load('en_core_web_sm')
doc1 = nlp(u'/content/gdrive/MyDrive/project/sum.txt')
doc2 = nlp(u'/content/gdrive/MyDrive/project/Contextual embeddings summary.txt')
n  = (doc1.similarity(doc2))
print (n*100)

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


94.20991285532648


<ipython-input-18-d0e9ea13c13d>:5: UserWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.
  n  = (doc1.similarity(doc2))


In [ ]:
import spacy
nlp = spacy.load('en_core_web_sm')
doc1 = nlp(u'/content/gdrive/MyDrive/project/sum2.0.txt')
doc2 = nlp(u'/content/gdrive/MyDrive/project/Contextual embeddings summary.txt')
n  = (doc1.similarity(doc2))
print (n*100)

95.48153854870543


<ipython-input-19-e8a93bb7713f>:5: UserWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.
  n  = (doc1.similarity(doc2))


In [ ]:
import spacy
nlp = spacy.load('en_core_web_sm')
doc1 = nlp(u'/content/gdrive/MyDrive/project/sum3.0.txt')
doc2 = nlp(u'/content/gdrive/MyDrive/project/Contextual embeddings summary.txt')
n  = (doc1.similarity(doc2))
print (n*100)

93.44686882194137


<ipython-input-20-01c260ad6c68>:5: UserWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.
  n  = (doc1.similarity(doc2))
